# SRT Data Preprocessing

In [ ]:
'''
{"messages": [{"role": "system", "content": "You are teaching assistant for Machine Learning. You should help to user to answer on his question."}, {"role": "user", "content": "What is machine learning?"}, {"role": "assistant", "content": "'Tis but the art of teaching machines to think, to learn from data most fine, and decisions to link."}]}
{"messages": [{"role": "system", "content": "You are teaching assistant for Machine Learning. You should help to user to answer on his question."}, {"role": "user", "content": "How doth neural networks work?"}, {"role": "assistant", "content": "They're like the brains of yon digital beast, with layers many, and nodes that cease."}]}
'''

In [ ]:
import json
import os
def convert_data_format(file_path, mode):
    '''
    Extract document and metadata from iaq json files
    '''
    if mode == '教材':
        # Load the JSON data from the file
        with open(file_path) as file:
            data = json.load(file)
        
        # Extract the 'parent' and 'text' values, merge them into 'knowledge', and add to the knowledge_list
        meta_data = {}
        meta_data['source'] = file_path.split('/')[-1].split('.')[0]
        knowledge_list = [{"messages":[ {"role":"system", "content": f"你很擅长处理保荐代表人考试的试题，我给你一个保荐代表人教材中的章节、考点，请你编写其对应的具体内容\n。"}, {"role":"user", "content":f"请根据以下指定章节及考点：{item['parent']}，从该考点的子考点中选取一个进行详细的内容编写。"} ,{"role": "assistant", "content": f"{item['parent']} \n{item['text']}"}]} 
                            for item in data.values() if 'parent' in item.keys() and item['parent'] != '']
        if '财务' in file_path:
            meta_data['class'] = '财务'
        elif '法规' in file_path:
            meta_data['class'] = '法规'
        return knowledge_list, meta_data
    
    elif mode == '例题':
        with open(file_path) as file:
            data = json.load(file)
        meta_data = {}
        meta_data['source'] = file_path.split('/')[-1].split('.')[0]
        knowledge_list = []
        knowledge_list.extend([ {"text": f"章节：{meta_data['source']}\n  题型：{item['instruction']}\n 问题：{item['input']} \n 答案：{item['output']} \n 解析：{item['CoT']}"}
                                 for item in data.values() if 'instruction' in item.keys()  and 'input'  
                                 in item.keys() and 'output'  in item.keys() and 'CoT'  in item.keys()]) 
        return knowledge_list, meta_data

# SyllabusQA

In [ ]:
import pandas as pd
csv_file_path = '/home/wangqi/Projects/r-r-rewriter/data/SyllabusQA/data/dataset_split/train.csv'
df = pd.read_csv(csv_file_path)
first_row = df.iloc[1]
# for column in df.columns:
#     print(f"{column}: {first_row[column]}")
print(first_row["question"])
print(first_row["answer"])

In [3]:
import os
import json

# 定义文件夹路径
folder_path = '/home/wangqi/Projects/r-r-rewriter/data/SyllabusQA/syllabi/syllabi_redacted/text'

# 初始化一个列表来存储所有文件的内容
texts = []

# 遍历文件夹中的所有文件
for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        file_path = os.path.join(folder_path, filename)
        
        with open(file_path, 'r', encoding='iso-8859-1') as file:
            content = file.read()
        
        texts.append({"text": filename + content})

# 将列表保存为JSON文件
json_file_path = '/home/wangqi/Projects/r-r-rewriter/data/SyllabusQA/texts.json'
with open(json_file_path, 'w', encoding='utf-8') as json_file:
    json.dump(texts, json_file, ensure_ascii=False, indent=4)

print(f"所有txt文件的内容已成功保存到{json_file_path}")

所有txt文件的内容已成功保存到/home/wangqi/Projects/r-r-rewriter/data/SyllabusQA/texts.json


# FintextQA

In [ ]:
import json

file_path = '/home/wangqi/Projects/LLM_Examinee/data/FintextQA/fin_dataset_test.json'

# Read the content of the JSON file
with open(file_path, 'r') as file:
    content = json.load(file)

text_list = []

for item in content:
    # Extract text from 'questions'
    # if 'questions' in item and isinstance(item['questions'], list):
    #     for question in item['questions']:
    #         if 'question' in question:
    #             text_list.append({"text" : question['question']})
    if 'paragraphs' in item and isinstance(item['paragraphs'], list):
        for paragraph in item['paragraphs']:
            if 'text' in paragraph:
                text_list.append({"text" :paragraph['text']})
    if 'table' in item and isinstance(item['table']['table'], list):
        merged_text = '\n'.join(['\t'.join(row) for row in item['table']['table']])
        text_list.append({"text" : merged_text})

json_file_path = '/home/wangqi/Projects/LLM_Examinee/data/FintextQA/texts.json'
with open(json_file_path, 'w', encoding='utf-8') as json_file:
    json.dump(text_list, json_file, ensure_ascii=False, indent=4)

print(f"所有json文件的内容已成功保存到{json_file_path}")

# Data Splitter